<a href="https://colab.research.google.com/github/FernandoBRdgz/diplomado_ds_mod4/blob/main/4.4%20Modelado%20Secuencial/4.4.1%20Procesamiento%20del%20lenguaje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#spaCy

[spaCy](https://spacy.io/) es una librería de Python de código abierto que analiza y "comprende" grandes volúmenes de texto. Hay modelos separados disponibles que se adaptan a idiomas específicos (inglés, español, francés, alemán, etc.).

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
text = 'Tesla is looking at buying U.S. startup for $6 million'

In [ ]:
doc = nlp(text)
type(doc)

spacy.tokens.doc.Doc

In [ ]:
for token in doc:
    print(token.text, token.pos_, token.dep_, "\n")

Tesla PROPN nsubj 

is AUX aux 

looking VERB ROOT 

at ADP prep 

buying VERB pcomp 

U.S. PROPN compound 

startup NOUN dobj 

for ADP prep 

$ SYM quantmod 

6 NUM compound 

million NUM pobj 



Esto no parece muy fácil de usar, pero de inmediato vemos que suceden algunas cosas interesantes:
1. Se reconoce que Tesla es un nombre propio, no solo una palabra al comienzo de una oración.
2. U.S. se mantiene unido como una sola entidad (lo llamamos "token")

A medida que profundicemos en spaCy, veremos qué significa cada una de estas abreviaturas y cómo se derivan.

## Tokenización

El primer paso en el procesamiento de texto es dividir todas las partes componentes (palabras y puntuación) en tokens. Estos tokens se anotan dentro del objeto `Doc` para contener información descriptiva.

In [ ]:
doc2 = nlp("Tesla isn't looking  into startups anymore.")

for token in doc2:
    print(token.text, token.pos_, token.dep_, "\n")

Tesla PROPN nsubj 

is AUX aux 

n't PART neg 

looking VERB ROOT 

  SPACE dep 

into ADP prep 

startups NOUN pobj 

anymore ADV advmod 

. PUNCT punct 



Observe cómo `isn't` se ha dividido en dos tokens. spaCy reconoce tanto el verbo raíz `is` como la negación adjunta. Observe también que tanto el espacio en blanco extendido como el punto al final de la oración tienen asignados sus propios tokens.

Es importante tener en cuenta que aunque `doc2` contiene información procesada sobre cada token, también conserva el texto original:

In [ ]:
doc2

Tesla isn't looking  into startups anymore.

In [ ]:
doc2[0]

Tesla

In [ ]:
type(doc2[0])

spacy.tokens.token.Token

## Etiquetado Gramatical (Part-of-Speech Tagging)

Para obtener una lista completa de etiquetas POS, visite https://universaldependencies.org/u/pos/

In [ ]:
doc2[0].pos_

'PROPN'

## Dependencias
También analizamos las dependencias sintácticas asignadas a cada token. `Tesla` se identifica como `nsubj` o el ***sujeto nominal*** de la oración.

Se puede encontrar una buena explicación de las dependencias escritas [aquí](https://nlp.stanford.edu/software/dependencies_manual.pdf)

In [ ]:
doc2[0].dep_

'nsubj'

Para ver el nombre completo de una etiqueta, use `spacy.explain(tag)`

In [ ]:
spacy.explain('PROPN')

'proper noun'

In [ ]:
spacy.explain('nsubj')

'nominal subject'

## Atributos de tokens adicionales

|Tag|Descripción|doc2[0].tag|
|:------|:------:|:------|
|`.text`|El texto original de la palabra<!-- .element: style="text-align:left;" -->|`Tesla`|
|`.lemma_`|La forma básica de la palabra|`tesla`|
|`.pos_`|La etiqueta gramatical simple (POS)|`PROPN`/`proper noun`|
|`.tag_`|La etiqueta gramatical detallada|`NNP`/`noun, proper singular`|
|`.shape_`|La forma de la palabra: mayúsculas, puntuación, dígitos|`Xxxxx`|
|`.is_alpha`|¿El token es un carácter alfabético?|`True`|
|`.is_stop`|¿El token es parte de una lista de stopwords, es decir, de las palabras más comunes del idioma?|`False`|

In [ ]:
# Lemas (la forma base de la palabra):
print(doc2[3].text)
print(doc2[3].lemma_)

looking
look


In [ ]:
# Parts-of-Speech Tagging:
print(doc2[3].pos_)
print(doc2[3].tag_ + ' / ' + spacy.explain(doc2[3].tag_))

VERB
VBG / verb, gerund or present participle


In [ ]:
# Formas de la palabra:
print(doc2[0].text+': '+doc2[0].shape_)
print(doc[5].text+' : '+doc[5].shape_)

Tesla: Xxxxx
U.S. : X.X.


In [ ]:
# Valores booleanos:
print(doc2[0].is_alpha)
print(doc2[0].is_stop)

True
False


## Spans (tramos)

A veces puede ser difícil trabajar con objetos grandes de un `Doc`. Un *span* es una porción del objeto `Doc` de la forma `Doc[start:stop]`.

In [ ]:
doc3 = nlp('Although commmonly attributed to John Lennon from his song "Beautiful Boy", \
the phrase "Life is what happens to us while we are making other plans" was written by \
cartoonist Allen Saunders and published in Reader\'s Digest in 1957, when Lennon was 17.')

In [ ]:
life_quote = doc3[16:30]
print(life_quote)

"Life is what happens to us while we are making other plans"


In [ ]:
type(life_quote)

spacy.tokens.span.Span

## Oraciones

Ciertos tokens dentro de un objeto `Doc` también pueden recibir una etiqueta de "Comienzo de oración".

In [ ]:
doc4 = nlp('This is the first sentence. This is another sentence. This is the last sentence.')

In [ ]:
for sent in doc4.sents:
    print(sent, "\n")

This is the first sentence. 

This is another sentence. 

This is the last sentence. 



In [ ]:
doc4[6].is_sent_start

True

## Conteo de Tokens

Los objetos `Doc` tienen un número determinado de tokens haciendo posible su conteo.

In [ ]:
# Se crea una cadena de texto que incluye comillas de apertura y cierre

mystring = '"We\'re moving to L.A.!"'
print(mystring)

"We're moving to L.A.!"


In [ ]:
doc5 = nlp(mystring)

for token in doc5:
    print(token.text, end=' | ')

" | We | 're | moving | to | L.A. | ! | " | 

In [ ]:
len(doc5)

8

# Entidades Nombradas

Las entidades nombradas agregan otra capa de contexto. El modelo de lenguaje reconoce que ciertas palabras son nombres de organizaciones mientras que otras son ubicaciones, y otras combinaciones se relacionan con dinero, fechas, etc. Las entidades nombradas son accesibles a través de la propiedad `ent` de un objeto `Doc`.

In [ ]:
doc6 = nlp('Apple will build a factory in Hong Kong for $6 million')

for token in doc6:
    print(token.text, end=' | ')

print('\n----')

for ent in doc6.ents:
    print(ent.text+' - '+ent.label_+' - '+str(spacy.explain(ent.label_)))

Apple | will | build | a | factory | in | Hong | Kong | for | $ | 6 | million | 
----
Apple - ORG - Companies, agencies, institutions, etc.
Hong Kong - GPE - Countries, cities, states
$6 million - MONEY - Monetary values, including unit


Observe cómo se combinan dos tokens para formar la entidad 'Hong Kong', y cómo se combinan tres tokens para formar la entidad monetaria: '$6 millones'

In [ ]:
len(doc6.ents)

3

# Visualizadores incorporados

spaCy incluye una herramienta de visualización integrada llamada **displaCy**.

Para obtener más información, visite https://spacy.io/usage/visualizers

In [ ]:
from spacy import displacy

doc7 = nlp('Apple is going to build a U.K. factory for $6 million.')
displacy.render(doc7, style='dep', jupyter=True, options={'distance': 110})

El argumento opcional `'distance'` establece la distancia entre tokens. Si la distancia se hace demasiado pequeña, el texto que aparece debajo de las flechas cortas puede quedar demasiado comprimido para leer.

In [ ]:
# Visualizando entidades nombradas

doc8 = nlp('Over the last quarter Apple sold nearly 20 thousand iPods for a profit of $6 million.')
displacy.render(doc8, style='ent', jupyter=True)

## Lematización

A diferencia de el stemming, la lematización va más allá de la reducción de palabras y considera el vocabulario completo de un idioma para aplicar un *análisis morfológico* a las palabras. El lema de `'was'` es `'be'` y el lema de `'mice'` es `'mouse'`. Además, el lema de `'meeting'` podría ser `'meet'` o `'meeting'` dependiendo de su uso en una oración.

In [ ]:
doc9 = nlp(u"I am a runner running in a race because I love to run since I ran today")

for token in doc9:
    print(token.text, '\t\t', token.pos_, '\t\t', token.lemma, '\t\t', token.lemma_)

I 		 PRON 		 4690420944186131903 		 I
am 		 AUX 		 10382539506755952630 		 be
a 		 DET 		 11901859001352538922 		 a
runner 		 NOUN 		 12640964157389618806 		 runner
running 		 VERB 		 12767647472892411841 		 run
in 		 ADP 		 3002984154512732771 		 in
a 		 DET 		 11901859001352538922 		 a
race 		 NOUN 		 8048469955494714898 		 race
because 		 SCONJ 		 16950148841647037698 		 because
I 		 PRON 		 4690420944186131903 		 I
love 		 VERB 		 3702023516439754181 		 love
to 		 PART 		 3791531372978436496 		 to
run 		 VERB 		 12767647472892411841 		 run
since 		 SCONJ 		 10066841407251338481 		 since
I 		 PRON 		 4690420944186131903 		 I
ran 		 VERB 		 12767647472892411841 		 run
today 		 NOUN 		 11042482332948150395 		 today


En la oración anterior, `running`, `run` y `ran` apuntan al mismo lema `run`.

## Función para mostrar lemas

Dado que la celda anterior está escalonada, es difícil de leer, por lo que se escribe una función que muestre la información que queremos de forma más clara.

In [ ]:
def show_lemmas(text):
    for token in text:
        print(f'{token.text:{12}} {token.pos_:{6}} {token.lemma:<{22}} {token.lemma_}')

In [ ]:
show_lemmas(doc9)

I            PRON   4690420944186131903    I
am           AUX    10382539506755952630   be
a            DET    11901859001352538922   a
runner       NOUN   12640964157389618806   runner
running      VERB   12767647472892411841   run
in           ADP    3002984154512732771    in
a            DET    11901859001352538922   a
race         NOUN   8048469955494714898    race
because      SCONJ  16950148841647037698   because
I            PRON   4690420944186131903    I
love         VERB   3702023516439754181    love
to           PART   3791531372978436496    to
run          VERB   12767647472892411841   run
since        SCONJ  10066841407251338481   since
I            PRON   4690420944186131903    I
ran          VERB   12767647472892411841   run
today        NOUN   11042482332948150395   today


In [ ]:
doc10 = nlp("I saw eighteen mice today!")
show_lemmas(doc10)

I            PRON   4690420944186131903    I
saw          VERB   11925638236994514241   see
eighteen     NUM    9609336664675087640    eighteen
mice         NOUN   1384165645700560590    mouse
today        NOUN   11042482332948150395   today
!            PUNCT  17494803046312582752   !


Observe que el lema de `saw` es `see`, `mice` es la forma plural de `mouse` y, sin embargo, `eighteen` es su propio número, *no* una forma expandida de `eight`.

In [ ]:
doc11 = nlp("I am meeting him tomorrow at the meeting.")
show_lemmas(doc11)

I            PRON   4690420944186131903    I
am           AUX    10382539506755952630   be
meeting      VERB   6880656908171229526    meet
him          PRON   1655312771067108281    he
tomorrow     NOUN   3573583789758258062    tomorrow
at           ADP    11667289587015813222   at
the          DET    7425985699627899538    the
meeting      NOUN   14798207169164081740   meeting
.            PUNCT  12646065887601541794   .


Aquí el lema `meeting` está determinado por el contexto de la oración.

## Stop Words

Palabras como `a` y `the` aparecen con tanta frecuencia que no requieren un etiquetado tan exhaustivo como los sustantivos, los verbos y los modificadores. Las llamamos stopwords o *palabras vacías* y se pueden filtrar del texto que se va a procesar. spaCy tiene una lista integrada de palabras vacías en varios idiomas.

In [ ]:
# Conjunto de palabras vacías predeterminadas de spaCy (recuerde que los conjuntos no tienen un orden):
print(nlp.Defaults.stop_words)

{'why', 'this', '’ll', 'for', 'himself', 'not', 'therefore', 'toward', 'you', 'seems', 'behind', 'seemed', '’m', 'almost', 'amongst', 'sixty', 'thereafter', 'their', 'anything', 'we', 'has', 'when', 'ever', 'each', 'becoming', 'mine', 'really', 'in', 'because', 'n‘t', 'is', 'during', 'doing', 'three', 'becomes', 'of', 'were', 'often', 'am', 'could', 'any', 'yours', 'except', 'front', 'afterwards', 'none', 'call', 'per', 'many', '‘ll', 'bottom', 'else', 'always', 'so', 'nor', 'though', 'upon', 'would', 'few', 'indeed', 'fifteen', 'which', 'i', 'and', 'who', 'at', 'again', 'the', 'too', 'a', 'from', 'as', 'whereas', 'yourself', 'ours', 'beforehand', 'ten', 'wherein', 'against', 'that', 'perhaps', 'to', 'via', 'have', 'over', 'does', 'used', 'neither', 'several', 'across', 'every', 'well', 'moreover', 'only', 'besides', 'take', 'part', 'elsewhere', '‘m', 'hereafter', 'nevertheless', 'put', 'between', 'within', 'around', 'such', 'everyone', 'no', 'further', '‘d', 'nobody', 'cannot', 'where

In [ ]:
len(nlp.Defaults.stop_words)

326

In [ ]:
# Para ver si una palabra es una stopword

In [ ]:
nlp.vocab['myself'].is_stop

True

In [ ]:
nlp.vocab['mystery'].is_stop

False

## Para agregar una stopword

Puede haber ocasiones en las que desee agregar una palabra vacía al conjunto predeterminado. Tal vez decidas que `'btw'` (abreviatura común para "*by the way*") debe considerarse una palabra vacía.

In [ ]:
# Agregue la palabra al conjunto de palabras vacías. ¡Use minúsculas!
nlp.Defaults.stop_words.add('btw')

# Establece la etiqueta stop_word en el lexema
nlp.vocab['btw'].is_stop = True

In [ ]:
len(nlp.Defaults.stop_words)

327

In [ ]:
nlp.vocab['btw'].is_stop

True

## Para eliminar una stopword

Alternativamente, puede decidir que `'beyond'` no debe considerarse una palabra vacía.

In [ ]:
# Elimina la palabra del conjunto de palabras vacías
#nlp.Defaults.stop_words.remove('beyond')

# Elimina la etiqueta stop_word del lexema
nlp.vocab['beyond'].is_stop = False

In [ ]:
len(nlp.Defaults.stop_words)

327

In [ ]:
nlp.vocab['beyond'].is_stop

False